In [39]:
# 재시작 없이 import 파일 자동 적용
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
import os
import h5py
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import torch
from torch.utils.data import DataLoader

# from config import CONFIG
from data_factory import collate_fn
from data_loader import PublicTest
from classification import Exp_Classification

### Checkpoints 및 CONFIG file 경로 설정

In [129]:
import importlib.util

test_folder_name = 'classification_T#01_Medformer_K-Medicon_bs8_sl96_lr0.0001_pl96_dm128_nh8_el6_df256_fc1_ebtimeF_Exp_seed41'
test_config_path = f"./checkpoints/{test_folder_name}/config_T#01.py"

# 모듈 이름 생성
module_name = os.path.splitext(os.path.basename(test_config_path))[0]

# 모듈 로드
spec = importlib.util.spec_from_file_location(module_name, test_config_path)
config_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config_module)

# CONFIG 변수 사용
CONFIG = config_module.CONFIG  # config_T#01.py 파일 내의 CONFIG 변수

In [130]:
print(CONFIG)

{'seed': 41, 'is_training': 1, 'model_id': 'T#01', 'task_name': 'classification', 'model': 'Medformer', 'monitor': 'vali_loss', 'data': 'K-Medicon', 'root_path': './dataset/', 'signal_hz': 250, 'seq_len': 96, 'pred_len': 96, 'enc_in': 7, 'c_out': 7, 'd_model': 128, 'n_heads': 8, 'e_layers': 6, 'd_ff': 256, 'factor': 1, 'dropout': 0.1, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'no_inter_attn': False, 'patch_len_list': '2,4,8,8,16,16,16,16,32,32,32,32,32,32,32,32', 'single_channel': False, 'augmentations': 'jitter0.2,scale0.2,drop0.5', 'num_workers': 0, 'itr': 1, 'train_epochs': 100, 'batch_size': 8, 'patience': 10, 'learning_rate': 0.0001, 'des': 'Exp', 'loss': 'CrossEntropy', 'lradj': 'type1', 'swa': True, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3'}


In [131]:
seed = CONFIG['seed']
    
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [132]:
setting = "{}_{}_{}_{}_bs{}_sl{}_lr{}_pl{}_dm{}_nh{}_el{}_df{}_fc{}_eb{}_{}_seed{}".format(
                CONFIG['task_name'],
                CONFIG['model_id'],
                CONFIG['model'],
                CONFIG['data'],
                CONFIG['batch_size'],
                CONFIG['seq_len'],
                CONFIG['learning_rate'],
                CONFIG['pred_len'],
                CONFIG['d_model'],
                CONFIG['n_heads'],
                CONFIG['e_layers'],
                CONFIG['d_ff'],
                CONFIG['factor'],
                CONFIG['embed'],
                CONFIG['des'],
                CONFIG['seed'],
            )

exp = Exp_Classification(CONFIG)

Use GPU: cuda:0


In [133]:
model_path = f"./checkpoints/{test_folder_name}/checkpoint.pth"

exp.swa_model.load_state_dict(torch.load(model_path))
exp.swa_model.eval()

AveragedModel(
  (module): Model(
    (enc_embedding): ListPatchEmbedding(
      (value_embeddings): ModuleList(
        (0): CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 2), stride=(1, 2), bias=False, padding_mode=circular)
        )
        (1): CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 4), stride=(1, 4), bias=False, padding_mode=circular)
        )
        (2-3): 2 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 8), stride=(1, 8), bias=False, padding_mode=circular)
        )
        (4-7): 4 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 16), stride=(1, 16), bias=False, padding_mode=circular)
        )
        (8-15): 8 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 32), stride=(1, 32), bias=False, padding_mode=circular)
        )
      )
      (position_embedding): PositionalEmbedding()
      

In [134]:
testset = PublicTest("./dataset/Signal_Test_Public.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [135]:
device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

100%|██████████| 300/300 [01:16<00:00,  3.93it/s]

(300, 2)


In [136]:
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
# auroc = roc_auc_score(trues, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")
print()
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

F1 score: 0.6511627906976744
AUROC score: 0.8635851851851852
MCC score: 0.5225275670408928
CPI score: 0.6399507774911612

0개수: 203
1개수: 97


In [137]:
# df = pd.DataFrame({"0":probs[:,0], "1":probs[:,1]})
df = pd.DataFrame({"Probability":probs[:,1]})

save_xlsx = pd.ExcelWriter(f"./RTFACT_241001_Public_Possibility_{CONFIG['model_id']}.xlsx")
df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환
save_xlsx.save() #xlsx 파일로 저장

In [128]:
predictions[15]

0